In [1]:
names = ["Ling",
"Shang the Monk",
"Kate",
"Marcus",
"Ironclad",
"Hong-Joo",
"Fireguard",
"Kibo",
"Helga",
"Sarge",
"Azuma",
"Midnight",
"Yukka and Shade",
"Jack Bulwark",
"Jet",
"Emperor",
"Lynx",
"Monkey King",
"Cobra"
        ]
letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
cols = letters.copy()
for i in range(len(letters)):
    for j in range(len(letters)):
        cols.append(letters[i] + letters[j])
        
skip = ["11.0", "12.0", "13.0"]

In [2]:
import xlrd
import xlsxwriter
import pandas as pd
import openpyxl
from openpyxl import workbook
from openpyxl import load_workbook

In [3]:
wb = load_workbook("BalanceHeroes/BalanceHeroes.xlsx")

In [4]:
#def WriteNames(sheet, prows, pcols):
#    for name in names:
#        wb[name].insert_rows(prows + 1 + p[names.index(name)] - 3, len(names) + 5)
#    for name in names:
#        for i in range(len(names)):
#            wb[name].cell(row = prows + i + 1 + p[names.index(name)], column = pcols + 1).value = names[i]

def CopyWinrates():
    for name in names:
        sheet = wb[name]
        for i in range(len(names)):
            oldName = sheet.cell(row = i + 1 + p[names.index(name)], column = 1).value
            sheet.cell(row = names.index(oldName) + 1 + p2[names.index(name)], column = 1).value = oldName
            for j in range(9):
                oldWinrate = sheet.cell(row = i + 1 + p[names.index(name)], column = j*2 + 2).value
                oldDif = sheet.cell(row = i + 1 + p[names.index(name)], column = j*2 + 3).value
                sheet.cell(row = names.index(oldName) + 1 + p2[names.index(name)], column = j*2 + 2).value = oldWinrate
                sheet.cell(row = names.index(oldName) + 1 + p2[names.index(name)], column = j*2 + 3).value = oldDif
    
def OperateHero(sheet, heroName, startRow, prows, pcols):
    strong = 0
    weak = 0
    for row in reversed(range(1, startRow + 1)):
        if sheet.cell(row, 1).value != heroName or row == 1:
            break             
        winRate = sheet.cell(row, 4).value
        oldWinRate = wb[heroName].cell(row = names.index(sheet.cell(row, 2).value) + 1 + p2[names.index(heroName)], column = pcols + 1).value
        if oldWinRate == None:
            oldWinRate = 0
        if sheet.cell(row, 3).value >= 30:
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 1).value = winRate
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 2).value = winRate - oldWinRate 
        else:
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 1).value = 0
            wb[heroName].cell(row = prows + names.index(sheet.cell(row, 2).value) + 1 + p[names.index(heroName)], column = pcols + 2).value = 0 
    wb[heroName].cell(row = prows + names.index(heroName) + 1 + p[names.index(heroName)], column = pcols + 1).value = "0"

def OperateCategory(sheet, catName, startRow, prows, pcols):
    i = 0
    hero = ""
    for row in reversed(range(1, startRow)):
        if sheet.cell(row, 0).value != catName:
            break        
        if hero != sheet.cell(row, 1).value:
            hero = sheet.cell(row, 1).value
            OperateHero(sheet, hero, row, prows, pcols)
            i = i + 1
        
def DoBalanceTable(sheet, padding):
    CopyWinrates()
    categories = []
    i = 0
    catName = ""
    for row in reversed(range(1, sheet.nrows)):
        if catName != sheet.cell(row, 0).value:
            categories.append(row)
            catName = sheet.cell(row, 0).value
            if not str(catName) in skip:
                OperateCategory(sheet, catName, row, padding, 1 + 2*i)
                i = i + 1
    return categories

def FindPaddings(book):
    p = []
    for sheet in wb.worksheets:
        i = 0
        for cell in sheet['A']:
            if cell.value == "#WINRATES":
                p.append(i + 4)
                break
            i = i + 1
    return p

def FindPaddings2(book):
    p2 = []
    for sheet in wb.worksheets:
        i = 0
        for cell in sheet['A']:
            if cell.value == "#WINRATES_2":
                p2.append(i + 4)
                break
            i = i + 1
    return p2

In [5]:
p = FindPaddings(wb)
p2 = FindPaddings2(wb)

In [6]:
# Винрейты

#wbRD = xlrd.open_workbook("BalanceSource/Balance1_2_10_7.xlsx")
#sheets = wbRD.sheets()
#DoBalanceTable(sheets[3], 0)

In [7]:
import pandas as pd
import os
import sys
import xlwt
import matplotlib.pyplot as plt

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from os import listdir
from os.path import isfile, join

In [8]:
class EhShasBiPoprogat:
    
    def __init__(self):
        text_file = open("kick_names.txt", "r", encoding='utf-8')
        self.names = text_file.read().split("\n")
        text_file = open("SameAttacks.txt", "r", encoding='utf-8')
        
        data = text_file.read()
        data = StringIO(data)

        self.df_same = pd.read_csv(data, sep=" ")
        self.df_same = self.df_same.apply(lambda x: x.str.lower())
        
        
    def is_shit(self, s):
        arr = s.split('_')
        arr = [x for x in arr if x.isdigit()]
        if len(arr)==0:
            return True
        if s.startswith("Shadow_Agile_Helmet_3"):
            return True
        return arr[-1]=='1'
    
    def is_move(self, s):
        return not s in self.names
    
    def is_not_move(self, s):
        return s in self.names
    
    def drop_shit(self, df):
        df = df[df["source"].map(self.is_shit)]
        df = df[~df["source"].str.startswith("lying_kick")]
        df = df[~df["source"].str.startswith("throw_forward_active")]
        df = df[~df["source"].str.startswith("throw_simple")]
        df = df.append(pd.DataFrame(data = {"source" : "legs", "amount" : df[df["source"].map(self.is_not_move)]["amount"].sum()} , index=[0]))
        df = df[~df["source"].apply(lambda x: any([x.startswith(name) for name in self.names]))]
        df = df[~df["source"].str.endswith("long")]
        df = df[~df["source"].str.startswith("dual_swords_heavy_slash")]
        df = df[~df["source"].apply(lambda x: any([x.startswith(name) for name in self.df_same["AddName"]]))]
        df = df[df.amount != 0]
        return df
    
    def operate_file(self, f, padding, worksheet, byHero):
        if byHero:
            file = "DMG/actionsDmg/byHero/above3000" + "/" + f
        else:
            file = "DMG/actionsDmg/byMatchup/above3000" + "/" + f
        fp = open(file, encoding='utf-8')

        data = fp.read()
        data = StringIO(data)

        dftmp = pd.read_csv(data, sep=" ")
        dfb = dftmp.loc["[B]"].reset_index(inplace=True, drop=True)
        dfbs = dftmp.loc["[BS]"].reset_index(inplace=True, drop=True)
        dfs = dftmp.loc["[S]"].reset_index(inplace=True, drop=True)
        df = dftmp.loc[dftmp.index.difference(["[B]", "[BS]", "[S]"])]
        df['amount'] = df['source']
        df['source'] = df.index
        df.reset_index(inplace=True, drop=True)
        df = df[~df["amount"].str.startswith("animation]")]
        df = df.astype({"amount": float})

        dfbuf = df
        for index, row in df.iterrows():
            add = 0
            tmp = self.df_same[self.df_same['AddName'].str.fullmatch(row['source'])]["MainName"]
            if not tmp.empty:
                add = dfbuf[dfbuf["source"].str.fullmatch(row['source'])].amount.values[0]
                dfbuf.loc[df.source == tmp.astype("string").values[0], 'amount'] += add
            if not row['source'] in self.df_same.values:
                dfbuf.at[index, 'amount'] = df.groupby(df["source"].str.startswith(row['source'][:-1]))["amount"].sum()[True]
        
        lying_kick_sum = df.groupby(df["source"].str.startswith("lying_kick"))["amount"].sum()[True]
        throws = df[(df.source == "throw_forward_active") | (df.source == "throw_simple")]["amount"].sum()
        df = self.drop_shit(dfbuf)
        df = df.append(pd.DataFrame([['lying_kick',lying_kick_sum], ['throws', throws]],columns=['source','amount'])).reset_index(drop=True)
        df.sort_values(by=['amount'], inplace=True, ascending=False)
        df.reset_index(inplace=True, drop=True)
        df
        plt.barh(df["source"], df["amount"])
        plt.title(label=f)
        plt.savefig("DMG/trash/" + f[:-4] + '.png', bbox_inches="tight")
        plt.clf()        
        img = openpyxl.drawing.image.Image('DMG/trash/'+ f[:-4] + '.png')
        img.anchor = cols[padding] + str(4)
        worksheet.add_image(img)
        

In [9]:
# Стата по дамагу
onlyfilesMain = [f for f in listdir("DMG/actionsDmg/byHero/above3000") if isfile(join("DMG/actionsDmg/byHero/above3000", f))]
onlyfilesOther = [f for f in listdir("DMG/actionsDmg/byMatchup/above3000") if isfile(join("DMG/actionsDmg/byMatchup/above3000", f))]

dmg = EhShasBiPoprogat()

writer = pd.ExcelWriter('DMG/Dmg_plots_New.xlsx', engine = 'xlsxwriter')
dfempty = pd.DataFrame([['','']],columns=['',''])

padding = 2

for f,i in zip(onlyfilesMain, range(len(onlyfilesMain))):
    dfempty.to_excel(writer, sheet_name = f)
    print(f)
    worksheet = wb[f[:-4]]
    dmg.operate_file(f, padding, worksheet, True)
    j = 0
    for ff,ii in zip(onlyfilesOther, range(len(onlyfilesOther))):
        if ff.startswith(f[:-4]):
            j = j + 1
            dmg.operate_file(ff, padding + 1 + j*6, worksheet, False)
        
writer.save()
writer.close()

Azuma.txt
Emperor.txt
Fireguard.txt
Helga.txt
Hong-Joo.txt
Ironclad.txt
Jack Bulwark.txt
Jet.txt
Kate.txt
Kibo.txt
Ling.txt
Lynx.txt
Marcus.txt
Midnight.txt
Monkey King.txt
Sarge.txt
Shang the Monk.txt
Yukka and Shade.txt


<Figure size 432x288 with 0 Axes>

In [10]:
wb.save("BalanceHeroes/BalanceHeroesNew.xlsx") 